In [2]:
# Import modules
import pandas as pd
import sklearn
import torch
from torch import nn
from sklearn.model_selection import train_test_split
torch.__version__

'1.13.1'

In [2]:
# Load the dataset and review the first 100 characters
with open("the-verdict.txt", "r",encoding="utf-8") as f:
    raw_text = f.read()
print("Length of text:", len(raw_text))
print(raw_text[:99])

Length of text: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
1+1

2